## RDD

#### Transformations:

https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

#### Actions

https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

In [1]:
import org.apache.spark.sql.SparkSession
// val spark = SparkSession.builder().appName("Sesion_3").master("local[*]").getOrCreate()
// val sc = spark.sparkContext

Intitializing Scala interpreter ...

Spark Web UI available at http://23LAP5CD0176GH7:4040
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1700672490517)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
import org.apache.spark.rdd.RDD
val pokemon_rdd: RDD[String] = sc.textFile("../../resources/data/csv/pokemon.csv")
    //.flatMap(line => (1 to 10).map(item => line + "," + item.toString))
pokemon_rdd.foreach(item => println(item))

260,SwampertMega Swampert,Water,Ground,635,100,150,110,95,110,70,3,False
#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
261,Poochyena,Dark,,220,35,55,35,30,30,35,3,False
262,Mightyena,Dark,,420,70,90,70,60,60,70,3,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
263,Zigzagoon,Normal,,240,38,30,41,30,41,60,3,False
264,Linoone,Normal,,420,78,70,61,50,61,100,3,False
265,Wurmple,Bug,,195,45,45,35,20,30,20,3,False
4,Charmander,Fire,,309,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,,405,58,64,58,80,65,80,1,False
6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
266,Silcoon,Bug,,205,50,35,55,25,25,15,3,False
267,Beautifly,Bug,Flying,395,60,70,50,100,50,65,3,False
6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
6,CharizardMega Chariza

import org.apache.spark.rdd.RDD
pokemon_rdd: org.apache.spark.rdd.RDD[String] = ../../resources/data/csv/pokemon.csv MapPartitionsRDD[1] at textFile at <console>:25


In [3]:
val clean_pokemon_rdd: RDD[Seq[Any]] = pokemon_rdd
    .map(line => line.split(","))
    .filter(array_str => array_str(0).matches("^[0-9]+$"))
    .map(array_str => Seq(array_str(0).toInt, array_str(1), array_str(2), array_str(3),
                          array_str(4).toInt, array_str(5).toInt, array_str(6).toInt, array_str(7).toInt,
                          array_str(8).toInt, array_str(9).toInt, array_str(10).toInt, array_str(11).toInt,
                          array_str(12).matches("True"))).persist()

clean_pokemon_rdd.foreach(item => println(item))

List(1, Bulbasaur, Grass, Poison, 318, 45, 49, 49, 65, 65, 45, 1, false)
List(2, Ivysaur, Grass, Poison, 405, 60, 62, 63, 80, 80, 60, 1, false)
List(3, Venusaur, Grass, Poison, 525, 80, 82, 83, 100, 100, 80, 1, false)
List(3, VenusaurMega Venusaur, Grass, Poison, 625, 80, 100, 123, 122, 120, 80, 1, false)
List(4, Charmander, Fire, , 309, 39, 52, 43, 60, 50, 65, 1, false)
List(5, Charmeleon, Fire, , 405, 58, 64, 58, 80, 65, 80, 1, false)
List(6, Charizard, Fire, Flying, 534, 78, 84, 78, 109, 85, 100, 1, false)
List(6, CharizardMega Charizard X, Fire, Dragon, 634, 78, 130, 111, 130, 85, 100, 1, false)
List(6, CharizardMega Charizard Y, Fire, Flying, 634, 78, 104, 78, 159, 115, 100, 1, false)
List(7, Squirtle, Water, , 314, 44, 48, 65, 50, 64, 43, 1, false)
List(8, Wartortle, Water, , 405, 59, 63, 80, 65, 80, 58, 1, false)
List(9, Blastoise, Water, , 530, 79, 83, 100, 85, 105, 78, 1, false)
List(9, BlastoiseMega Blastoise, Water, , 630, 79, 103, 120, 135, 115, 78, 1, false)
List(10, Cater

clean_pokemon_rdd: org.apache.spark.rdd.RDD[Seq[Any]] = MapPartitionsRDD[4] at map at <console>:28


In [5]:
// Cuantos pokemon legendarios existen?
val count_legendary: Int = clean_pokemon_rdd
    .map(array => (array(12), 1))
    .filter(item => item._1 == true)
    .map(item => item._2)
    .reduce((item_1, item_2) => item_1 + item_2)

println(count_legendary)

val count_legendary_2: Int = clean_pokemon_rdd
    .map(array => (array(12), 1))
    .reduceByKey((item_1, item_2) => item_1 + item_2)
    .filter(item => item._1 == true)
    .collect()(0)._2

print(count_legendary_2)

43
43

count_legendary: Int = 43
count_legendary_2: Int = 43


In [28]:
// Valor promedio de ataque por cada type_1
clean_pokemon_rdd
    .map(array => (array(2), array(5).toString.toFloat))
    .groupByKey()
    .mapValues(values => (values.sum, values.size))
    .mapValues(items => items._1 / items._2)
    .foreach(item => println(item))

println("+-+-+")

clean_pokemon_rdd
    .map(array => (array(2), array(5).toString.toFloat))
    .groupByKey()
    .map(values => (values._1, (values._2.sum / values._2.size)))
    .foreach(item => println(item))

println("+-+-+")

clean_pokemon_rdd
    .map(array => (array(2), (array(5).toString.toFloat, 1)))
    .reduceByKey((tuple_1, tuple_2) => (tuple_1._1 + tuple_2._1, tuple_1._2 + tuple_2._2))
    .map(results => (results._1, results._2._1 / results._2._2))
    .foreach(item => println(item))

(Grass,66.18868)
(Dark,65.333336)
(Poison,68.25)
(Fighting,63.294117)
(Fire,68.5)
(Bug,56.208332)
(Electric,58.78788)
(Ground,70.5)
(Water,71.52809)
(Rock,63.482758)
(Dragon,79.73684)
(Normal,78.62666)
(Psychic,70.38461)
(Steel,68.27778)
(Ice,75.0)
(Fairy,67.5)
(Ghost,69.52941)
+-+-+
(Grass,66.18868)
(Dark,65.333336)
(Poison,68.25)
(Fighting,63.294117)
(Fire,68.5)
(Bug,56.208332)
(Electric,58.78788)
(Ground,70.5)
(Water,71.52809)
(Rock,63.482758)
(Dragon,79.73684)
(Normal,78.62666)
(Psychic,70.38461)
(Steel,68.27778)
(Ice,75.0)
(Fairy,67.5)
(Ghost,69.52941)
+-+-+
(Grass,66.18868)
(Electric,58.78788)
(Dark,65.333336)
(Ground,70.5)
(Poison,68.25)
(Water,71.52809)
(Fighting,63.294117)
(Rock,63.482758)
(Fire,68.5)
(Dragon,79.73684)
(Bug,56.208332)
(Normal,78.62666)
(Psychic,70.38461)
(Steel,68.27778)
(Ice,75.0)
(Fairy,67.5)
(Ghost,69.52941)


In [37]:
// Pokemon con mejor score en Total POR cada TYPE 1 Y POR GENERACIÓN OMITIENDO LOS LEGENDARIOS
clean_pokemon_rdd
    .filter(array => array(12) == false)
    .map(array => ((array(2), array(11)), (array(1), array(4).toString.toInt)))
    .reduceByKey((item_1, item_2) => if (item_1._2 > item_2._2) item_1 else item_2)
    .foreach(item => println(item))

((Steel,2),(SteelixMega Steelix,610))
((Fire,1),(CharizardMega Charizard Y,634))
((Grass,4),(AbomasnowMega Abomasnow,594))
((Fighting,3),(MedichamMega Medicham,510))
((Fire,4),(Magmortar,540))
((Fighting,1),(Machamp,505))
((Normal,4),(LopunnyMega Lopunny,580))
((Poison,1),(Nidoking,505))
((Bug,3),(Ninjask,456))
((Normal,2),(Blissey,540))
((Fighting,2),(Hitmontop,455))
((Bug,2),(HeracrossMega Heracross,600))
((Ground,1),(Rhydon,485))
((Fire,3),(BlazikenMega Blaziken,630))
((Grass,2),(Meganium,525))
((Electric,1),(Jolteon,525))
((Poison,2),(Crobat,535))
((Fairy,4),(Togekiss,545))
((Ghost,2),(Misdreavus,435))
((Psychic,4),(GalladeMega Gallade,618))
((Rock,2),(TyranitarMega Tyranitar,700))
((Water,4),(Manaphy,600))
((Electric,2),(AmpharosMega Ampharos,610))
((Bug,4),(Yanmega,515))
((Fairy,2),(Granbull,450))
((Grass,5),(Serperior,528))
((Water,1),(GyaradosMega Gyarados,640))
((Water,2),(Kingdra,540))
((Dark,4),(Weavile,510))
((Ice,4),(Mamoswine,530))
((Water,3),(SwampertMega Swampert,635))


In [6]:
// Top 3 Pokemon con mejor score en Total POR cada TYPE 1 Y POR GENERACIÓN OMITIENDO LOS LEGENDARIOS
clean_pokemon_rdd
    .filter(array => array(12) == false)
    .map(array => ((array(2), array(11)), (array(1), array(4).toString.toInt)))
    .groupByKey()
    .mapValues(values => values.toSeq.sortBy(item => item._2)(Ordering[Int].reverse).take(3))
    .flatMapValues(results => results)
    .foreach(item => println(item))

((Normal,2),(Blissey,540))
((Fire,1),(CharizardMega Charizard X,634))
((Fire,1),(CharizardMega Charizard Y,634))
((Fire,1),(Arcanine,555))
((Normal,2),(Porygon2,515))
((Normal,2),(Ursaring,500))
((Steel,2),(SteelixMega Steelix,610))
((Fighting,2),(Hitmontop,455))
((Fighting,2),(Tyrogue,210))
((Steel,2),(Steelix,510))
((Steel,2),(Skarmory,465))
((Electric,1),(Jolteon,525))
((Electric,1),(Electabuzz,490))
((Electric,1),(Raichu,485))
((Grass,4),(AbomasnowMega Abomasnow,594))
((Grass,4),(Tangrowth,535))
((Grass,4),(Torterra,525))
((Grass,2),(Meganium,525))
((Grass,2),(Bellossom,490))
((Grass,2),(Jumpluff,460))
((Fighting,3),(MedichamMega Medicham,510))
((Fighting,3),(Hariyama,474))
((Fighting,3),(Medicham,410))
((Poison,2),(Crobat,535))
((Fire,4),(Magmortar,540))
((Fire,4),(Infernape,534))
((Fire,4),(Monferno,405))
((Ghost,3),(BanetteMega Banette,555))
((Ghost,3),(Banette,455))
((Ghost,3),(Dusclops,455))
((Fairy,4),(Togekiss,545))
((Fighting,1),(Machamp,505))
((Water,4),(Manaphy,600))
((Wa